In [2]:
from main import experiments

In [15]:
segments = ['urbana', 'calurosa', 'parachicos']
text_features = ['titulo', 'descripcion']
features = ['metroscubiertos', 'mejorciudad', 'antiguedad']
predict_column = 'precio'
carititud_column = 'carititud'


In [17]:
experiments(segments, text_features, features)

Error calculando metricas en segmento /True/False/False:
Mean Squared Logarithmic Error cannot be used when targets contain negative values.
Error calculando metricas en segmento /True/False/False:
Mean Squared Logarithmic Error cannot be used when targets contain negative values.
Error calculando metricas en segmento /True/False/False:
Mean Squared Logarithmic Error cannot be used when targets contain negative values.
Error calculando metricas en segmento /True/False/False:
Mean Squared Logarithmic Error cannot be used when targets contain negative values.
Error calculando metricas en segmento /True/False/False:
Mean Squared Logarithmic Error cannot be used when targets contain negative values.
Error calculando metricas en segmento /True/True/False:
Mean Squared Logarithmic Error cannot be used when targets contain negative values.
Error calculando metricas en segmento /True/True/False:
Mean Squared Logarithmic Error cannot be used when targets contain negative values.
Error calculand

/home/tomy/.pyenv/versions/3.6.9/envs/py3.6/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[Pipeline] ............... (step 3 of 3) Processing reg, total= 2.2min
Error calculando metricas en segmento /True/False/False:
Mean Squared Logarithmic Error cannot be used when targets contain negative values.
Error calculando metricas en segmento /True/False/False:
Mean Squared Logarithmic Error cannot be used when targets contain negative values.
Error calculando metricas en segmento /True/False/False:
Mean Squared Logarithmic Error cannot be used when targets contain negative values.
Error calculando metricas en segmento /True/False/False:
Mean Squared Logarithmic Error cannot be used when targets contain negative values.
Error calculando metricas en segmento /True/False/False:
Mean Squared Logarithmic Error cannot be used when targets contain negative values.
Error calculando metricas en segmento /True/True/False:
Mean Squared Logarithmic Error cannot be used when targets contain negative values.
Error calculando metricas en segmento /True/True/False:
Mean Squared Logarithmic Err

In [ ]:
X = np.sort(predictions['Villa'][1][:, 0]) #aproximado
Y = np.sort(predictions['Villa'][1][:, 1]) #real

plt.plot(X, color='red')
plt.plot(Y, color='blue')


plt.fill_between(np.arange(np.size(X)), X, Y, X>Y, color='pink')
plt.fill_between(np.arange(np.size(X)), X, Y, X<Y, color='lightblue')

In [18]:
train_df

NameError: name 'train_df' is not defined